In [1]:
import numpy as np
import pandas as pd

## Conjuntos 
Del modelo de optimización

| **Término** | **Descripción**                |
|------------|--------------------------------|
| $cacopios$ | Conjunto de centros de acopio  |
| $clientes$ | Conjunto de clientes           |
| $productos$ | Conjunto de productos          |

In [2]:
cacopios = 'CAcopios'
clientes = 'Clientes'
productos = 'Productos'

In [3]:
csv_base = './csv/'

routes = {
    # Capacidades de centros de acopio
    'cic': 'CapInvCA.csv',
    'cictest': 'CapInvCAtest.csv',
    # Costo de inventario del centro de acopio
    'cia' : 'CostoInvCA.csv',
    'ciatest' : 'CostoInvCAtest.csv',
    # Costo de transporte del centro de acopio al cliente
    'ctac' : 'CostoTransCAClie.csv',
    'ctactest' : 'CostoTransCAClietest.csv'
}

csv_config = {
    'delimiter' : ';',
    'decimal' : ','
}

## Centros de acopio

In [4]:
cacopios_df = pd.read_csv(csv_base + cacopios + 'test' + '.csv')
cacopios_df

,CAcopios
0,San Juan del Cesar
1,Riohacha
2,Maicao


## Clientes

In [5]:
clientes_df = pd.read_csv(csv_base + clientes + 'test' + '.csv')
clientes_df

,Clientes
0,Distribuidor1
1,Distribuidor2


## Productos

In [6]:
productos_df = pd.read_csv(csv_base + productos + '.csv')
productos_df.head()

,Productos
0,Queso duro
1,Queso blando


# Escenarios de prueba
Tomando demandas por cada centro de acopio, se definen distintos escenarios de posibles casos.

**Escenario 1**: Un centro de acopio sobrepasa la demanda del cliente
- El modelo puede asignar dicho centro de acopio para suplir con la demanda
- El centro de acopio debe proveer esa demanda

**Escenario 2**: Ningún centro de acopio cumple con toda la demanda del cliente
- El modelo debe dar una solución de varios centros de acopio
- La suma de la demanda por cada centro de acopio debe ser igual a la demanda

**Escenario 3**: Un centro de acopio tiene en stock exactamente la cantidad de la demanda de un cliente.
- El modelo puede tomar el centro de acopio que tiene en stock exactamente la demanda a cumplir

### Demandas
Generando demandas para cada cliente, se usarán como entrada para el modelo

In [7]:
demanda_clientes = clientes_df[clientes].drop_duplicates()

demandas = []

for client in demanda_clientes:
    demandas.append(20 * np.random.randint(10))

demanda_clientes = pd.DataFrame(demanda_clientes)

demanda_clientes['Demanda'] = pd.DataFrame(demandas)
demanda_clientes

,Clientes,Demanda
0,Distribuidor1,120
1,Distribuidor2,140


### Capacidades 
Obteniendo las capacidades de cada centro de acopio

In [8]:
cap_df = pd.read_csv(csv_base + routes['cictest'], **csv_config)

cap_df

,CAcopios,Productos,CapInvCA
0,San Juan del Cesar,Queso duro,50
1,San Juan del Cesar,Queso blando,320
2,Riohacha,Queso duro,60
3,Riohacha,Queso blando,90
4,Maicao,Queso duro,100
5,Maicao,Queso blando,0


#### Costos de Inventario
$$
  CInventario_t = \sum_{j \in J} \sum_{p \in P} CostoInvAcopio_{pjt} InvCA_{pjt} \qquad \forall_t \in T
$$

In [9]:
cia_df = pd.read_csv(csv_base + routes['ciatest'], **csv_config)
cia_df.head()

,CAcopios,Productos,CostoInvCA
0,San Juan del Cesar,Queso duro,1000
1,San Juan del Cesar,Queso blando,1500
2,Riohacha,Queso duro,500
3,Riohacha,Queso blando,800
4,Maicao,Queso duro,900


#### Costos de Transporte
$$
  CTransporte_t = \sum_{k \in K} \sum_{j \in J} \sum_{p \in P} CostoTransAcopioClie_{pjkt} \qquad \forall_t \in T
$$

In [10]:
ctac_df = pd.read_csv(csv_base + routes['ctactest'], **csv_config)
ctac_df.head()

,CAcopios,Clientes,CostoTransCAClie
0,San Juan del Cesar,Distribuidor1,68000
1,San Juan del Cesar,Distribuidor2,38000
2,Riohacha,Distribuidor1,21000
3,Riohacha,Distribuidor2,15000
4,Maicao,Distribuidor1,99000


## Esquema de representación
Se recibe la demanda $DemandaClie_{pkt}$ como parámetro de entrada.

Se representa la cantidad $n$ de centros de acopio como un vector, donde se asigna una cantidad $x_i$ para cada centro de acopio $J_i$, la cantidad asignada representa la demanda que va a ser suplida por ese centro de acopio.
$$
\begin{array} {|r|r|r|r|r|r|}
    \hline x_0 & x_1 & x_2 & x_3 & \cdots & x_n \\
    \hline
\end{array}
\quad \therefore \quad x_i = AC_{pjkt}
$$
Donde se aplican las restricciones:
$$
\begin{align*}
    \sum_{i=0}^{n} x_i &= DemandaClie_{pkt} \\
    x_i &\leq InvCA_{pjt}
\end{align*}
$$

In [11]:
X = np.zeros(cacopios_df.shape)
# Se toma la primera demanda como parámetro de entrada
demanda = demanda_clientes.loc[0, 'Demanda']

cap_queso = cap_df[cap_df[productos] == 'Queso duro']

for i in range(X.shape[0]):
    rand = np.random.randint(demanda)
    cap = cap_df.loc[i, 'CapInvCA']
        
    if i == X.size - 1 and demanda != 0:
        X[i] = demanda
        demanda -= demanda
    else:
        if rand > cap:
            X[i] = cap
            demanda -= cap
        else:
            X[i] = rand
            demanda -= rand
    
suma = X.sum()
    
print(X)
print(f'Suma de la distribución: {suma}')

[[50.]
 [34.]
 [36.]]
Suma de la distribución: 120.0


### Cálculo de costos en función objetivo

In [12]:
# Esta función calcula los costos por cada centro de acopio, y los anexa al nombre del centro de acopio, no es necesario mantenerla debido a que este cálculo se hace en la función objetivo, se eliminará pronto

def costos():
    sum_costos_ca = []

    for costo in range(X.shape[0]):
        ca = cacopios_df.loc[costo]
        delta = 0
        
        if X[costo] == 0:
            sum_costos_ca.append((ca.tolist()[0], 0.))
            continue
    
        costo_inv_ca = cia_df[(cia_df[cacopios] == ca[cacopios]) & (cia_df[productos] == 'Queso duro')]
        costo_trans_ca = ctac_df[(ctac_df[cacopios] == ca[cacopios]) & (ctac_df[clientes] == 'Distribuidor1')]
    
        delta += X[costo] * costo_inv_ca['CostoInvCA']
        delta += np.array(costo_trans_ca['CostoTransCAClie'])
    
        sum_costos_ca.append((ca.tolist()[0], delta.tolist()[0]))
        
    return sum_costos_ca

[('San Juan del Cesar', 118000.0), ('Riohacha', 38000.0), ('Maicao', 131400.0)]

## Función objetivo
$$
Min(F) = CInventario + CTransporte
$$

In [13]:
# Min(F(x))
def f(x):
    delta = 0
    print(f'{x} -> Suma: {x.sum()}')

    for idx in range(x.shape[0]):
        ca = cacopios_df.loc[idx]

        if x[idx] == 0.:
            continue

        # Costos de Inventario
        costo_inv_ca = cia_df[(cia_df[cacopios] == ca[cacopios]) & (cia_df[productos] == 'Queso duro')]
        # Costos de Transporte
        costo_trans_ca = ctac_df[(ctac_df[cacopios] == ca[cacopios]) & (ctac_df[clientes] == 'Distribuidor1')]

        delta += x[idx] * costo_inv_ca['CostoInvCA'].values[0]
        delta += np.array(costo_trans_ca['CostoTransCAClie'].values[0])

    print(f'Costo total: {delta}\n')
    return delta

## scipy

In [14]:
from scipy.optimize import dual_annealing
# from scipy.optimize import differential_evolution

from scipy.optimize import LinearConstraint

bounds = [(0, x_i) for x_i in cap_queso['CapInvCA']]
cons = {'constraints': LinearConstraint(np.ones(X.shape[0]), demanda, demanda)}
x0 = [x[0] for x in X]
    
result = dual_annealing(
    f,
    bounds=bounds,
    minimizer_kwargs=cons,
    maxiter=10,
    x0=np.array(x0)
)
print(result)

[50. 34. 36.] -> Suma: 120.0
Costo total: 287400.0

[15.79283698  9.79283698 51.79283698] -> Suma: 77.37851093709469
Costo total: 255302.80874967575

[39.89720101 29.27907825 11.27907825] -> Suma: 80.45535749569535
Costo total: 252687.91054859757

[39.29897656 58.6808538  60.6808538 ] -> Suma: 158.66068415343761
Costo total: 311252.1718740463

[42.23677661 29.27907825 11.27907825] -> Suma: 82.79493309557438
Costo total: 255027.4861484766

[39.89720101 39.56023239 11.27907825] -> Suma: 90.73651163652539
Costo total: 257828.4876190126

[39.89720101 29.27907825 49.93868321] -> Suma: 119.11496246233582
Costo total: 287481.555018574

[39.89720101 29.27907825 11.27907825] -> Suma: 80.45535749569535
Costo total: 252687.91054859757

[39.89720102 29.27907825 11.27907825] -> Suma: 80.45535751059651
Costo total: 252687.91056349874

[39.89720101 29.27907826 11.27907825] -> Suma: 80.45535751059651
Costo total: 252687.91055604815

[39.89720101 29.27907825 11.27907826] -> Suma: 80.45535751059651
Cost